In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import nltk

In [ ]:
from nltk.corpus import reuters

In [ ]:
print(reuters.fileids()[:10])

In [ ]:
len(reuters.categories())

In [ ]:
print(reuters.categories())

In [ ]:
## categories overlap on the Reuters corpus
print(reuters.categories('training/9865'))

In [ ]:
raw1 = reuters.raw('test/14826')
print(raw1)

In [ ]:
chars1 = [',', '"', ':', ')', '(', '\n', '>', '<', ';', "'s", "'", '&']
chars2 = ['. '] #['-'] ## Hong-kong vs far-reaching ?
def clean_text(x):
    for char in chars1:
        if char in x:
            x = x.replace(char, '')
    for char in chars2:
        if char in x:
            x = x.replace(char, ' ')
    return x
clean1 = clean_text(raw1)
print(clean1)

In [ ]:
clean1 = clean1.lower()
print(clean1)

In [ ]:
nltk.download('words')

In [ ]:
from nltk.corpus import words
## check if word exists
"would" in words.words()

In [ ]:
nltk.download('omw-1.4')

In [ ]:
nltk.download("wordnet") ## another way

from nltk.corpus import wordnet

wordnet.synsets("world")

In [ ]:
import re

In [ ]:
clean1 = re.sub(' +', ' ', clean1)
print(clean1)

In [ ]:
## tokenization
words1 = clean1.split(" ")
print(words1)

In [ ]:
article1 = nltk.Text(words1)
## how many words?
len(article1)

In [ ]:
article1.count("trade")

In [ ]:
article1.common_contexts(["japan", "u.s"])

In [ ]:
article1.dispersion_plot(["japan", "australia", 'u.s'])

In [ ]:
article1.concordance("japan")

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
cachedStopWords = stopwords.words('english')
print(cachedStopWords)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from matplotlib import rcParams


In [ ]:

## without stop words
wc1 = WordCloud(stopwords=None, max_words=25, background_color="white").generate(clean1)

rcParams["figure.figsize"] = (10,5)
plt.imshow(wc1)
plt.axis("off")
plt.show()

In [ ]:

## without stop words
wc1 = WordCloud(stopwords=None, max_words=25, background_color="white").generate_from_text(clean1)

rcParams["figure.figsize"] = (10,5)
plt.imshow(wc1)
plt.axis("off")
plt.show()

In [ ]:
## add some stopwords to the current list
cachedStopWords.append("said")
cachedStopWords.append("might")
## with stop words
wc2 = WordCloud(stopwords=cachedStopWords, max_words=25, background_color="white").generate_from_text(clean1)


rcParams["figure.figsize"] = (10,5)
plt.imshow(wc2)
plt.axis("off")
plt.show()

In [ ]:
porter = nltk.PorterStemmer()
lancaster = nltk.LancasterStemmer()
tokens = article1[0:10]

print(tokens)

print([porter.stem(t) for t in tokens])
print([lancaster.stem(t) for t in tokens])

In [ ]:
nltk.download('wordnet') ## wordnet is a rich dictionnary with definitions and synon # import these modules
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
print("rocks :", lemmatizer.lemmatize("rocks"))
print("corpora :", lemmatizer.lemmatize("corpora"))

# a denotes adjective in "pos"
print("better :", lemmatizer.lemmatize("better", pos ="a")) # v denotes verb in "pos"
print("is :", lemmatizer.lemmatize("is", pos ="v"))

In [ ]:
from nltk.corpus import wordnet as wn
## "synonym set", a collection of synonymous words (or "lemmas")
wn.synsets('motorcar')

In [ ]:
wn.synsets('car') ## ambiguous word

In [ ]:
i = '1'
print(wn.synset('car.n.0'+i).definition())
print(wn.synset('car.n.0'+i).lemma_names())

In [ ]:
## frequency distributions
fdist = nltk.FreqDist(article1)
print(fdist)

In [ ]:
## for most common words
fdist.most_common(10)

In [ ]:
## words count
fdist.plot(10, cumulative=True)

In [ ]:
## freqency of word lengths
fdist = nltk.FreqDist(len(w) for w in article1)
fdist.most_common()

In [ ]:
fdist.tabulate()

In [ ]:
from nltk import word_tokenize
import re

def tokenize(text, min_length=3):
    """
    A tokenizer typical used for classification
    """
    words = map(lambda word: word.lower(), word_tokenize(text))
    words = [word for word in words if word not in cachedStopWords]
    tokens = (list(map(lambda token: porter.stem(token), words)))
    p = re.compile('[a-zA-Z]+')
    filtered_tokens = list(filter(lambda token: p.match(token) and len(token) >= min_length, tokens))
    return filtered_tokens

In [ ]:
text = 'Stock futures soared Monday morning after Moderna (MRNA) became the latest m'
tokenize(text)

In [ ]:
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=tokenize)
## we take 10 documents from the Reuters dataset
docs = [reuters.raw(doc_id) for doc_id in reuters.fileids()[:10]]
docs_bow = vectorizer.fit_transform(docs)

In [ ]:
print(docs_bow.shape)
docs_bow.toarray()[1,]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer2 = TfidfVectorizer(tokenizer=tokenize)
docs_bow2 = vectorizer2.fit_transform(docs)

In [ ]:
print(docs_bow2.shape)
np.round(docs_bow2.toarray()[1,], 2)

In [ ]:
from textblob import TextBlob
blob = TextBlob(raw1)
print(blob.polarity)

In [ ]:
def loadSentimentDict(filename):
    myfile = open(filename, "r")
    lines = myfile.readlines()
    sent = [l.split(",")[0].strip().lower() for l in lines]
    return sent

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')

In [ ]:
data

In [ ]:
data['Positive'].unique()

In [ ]:
from nltk import sent_tokenize ## Notice: this is a sentence tokenizer
text = raw1
sentences = sent_tokenize(text)
print(len(sentences))
##
sentences[:2]

In [ ]:
print(tokenize(sentences[0]))

In [ ]:
sent_tfidf = vectorizer2.fit_transform(sentences).toarray()
print(sent_tfidf.shape)
np.round(sent_tfidf, 2)

In [ ]:
def score_sentences(x):
    sentenceValue = {}
    ## loop over the rows
    for i in range(x.shape[0]):
        total_score_per_sentence = np.sum(x[i,:])
        count_words_in_sentence = np.sum(x[i,:] > 0)
        sentenceValue[i] = total_score_per_sentence / count_words_in_sentence
    return sentenceValue
sent_scores = score_sentences(sent_tfidf)

In [ ]:
qtl = 0.9
sent_threshold = np.quantile(list(sent_scores.values()), qtl)
print(sent_threshold)
def generate_summary(sentences, scores, threshold):
    sentence_count = 0
    summary = ''
    for i in range(len(scores)):
        if scores[i] >= (threshold):
            summary += " \n\n " + sentences[i]
            sentence_count += 1
    return summary, sentence_count
summary, count = generate_summary(sentences, sent_scores, sent_threshold)
print(summary)

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence

# load tagger
classifier = TextClassifier.load('sentiment')

In [ ]:
# make example sentence
sentence = Sentence("Finnish steel maker Rautaruukki Oyj ( Ruukki ) said on July 7 , 2008 that it won a 9.0 mln euro ( $ 14.1 mln ) contract to supply and install steel superstructures for Partihallsforbindelsen bridge project in Gothenburg , western Sweden.")

# call predict
classifier.predict(sentence)

# check prediction
print(sentence)

In [ ]:
# Get all cik number available (intersection between our dataframe and the data on CRSP)
ciks_data = db.raw_sql(f'''
        select distinct comp_cik
        from crsp_q_mutualfunds.crsp_cik_map
        where comp_cik in ({ciks})
''')